In [3]:
##Use file.txt from this code to remove homopolymers from 5th column
import vcf
import pandas as pd

#chrs = [x for x in range(1,23,1)]
#chrs.append('X') ; chrs.append('Y')
Out=open('file.txt','w')
#VCF extracting the ref and alt alleles
for i in chrs:
#    Collecting alleles
    strfile= "/storage/szfeupe/Runs/GTEx_estr/STRs_No_Missing_genotypes/STR_filter.chr"+str(i)+".vcf.gz"
    STRs = vcf.Reader(filename=strfile)
    #print "Chr",i,' ...'
    for record in STRs:
        Data = [record.CHROM, record.POS, record.REF, record.ALT, record.INFO['PERIOD'], record.INFO['REFAC']]
        Out.write("\t".join([str(b) for b in Data])+'\n')
    print ("Done with chr",i,' ...')
Out.close()
see=pd.read_csv('file.txt', sep='\t', header=None)    
print ("done...")
see[2] 

Done with chr 1  ...
Done with chr X  ...
Done with chr Y  ...
done...


0                              GGTGGTGGTGGGGGCGGTGGGGGTGGTG
1                                              GTGTGTGTGTGT
2                                         GGTTTTTTTTTTTTTTT
3         TGTGTCTCCCTCTCTCTCTCTCTCTCTCTCTCTCATTTCTCTCTAT...
4                                            AAAAAAAAAAAAAA
5                                      CCATTAAAAAAAAAAAAAAA
6                                 AAAGTTTTTTTTTCTTTTTTTTTTT
7                               ATAAAAAATAAATAAATAAATAAAAAC
8                                             AAAAAAAAAAAAA
9                                           AAATAAATAAATAAA
10                                     TTTTTTTTTTTTTTTTTTTG
11                                       GCAAAAAAAAAAAAAAAA
12                AAATAAATAAATAAATAAATAAATAAATAAATTAAATAAAT
13                                          TTTTTTTTTTTTTTG
14                                AAAAAGAAAAAGAAAAAAAGAAAAA
15                       AAAAGCAAAACAAACAAACAAACAAAACAAAACA
16                        GGTGTGTGTGTGTG

In [ ]:
##This code will remove homopolymer and least polymorphic sitesfrom linear regression analysis output
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import vcf
import os

Input='/storage/szfeupe/Runs/GTEx_estr/Analysis_by_Tissue/Esophagus-Mucosa/SNP_Analysis/' #CHANGE HERE!!!
print(Input)
# STR size unit file
Reg='/storage/szfeupe/data/Regions_HipSTR_reference.hg19.bed'
regions=pd.read_csv(Reg, sep='\t', header=None)
                #ID addition
regions[1]= regions[1].astype('str')
regions['ID'] = regions[0]+'_'+ regions[1]
print('HipSTR regions: ',regions.shape)
#Remove homopolymers
poly=regions.loc[regions[3]!=1]
print('Non homopolymer regions: ',poly.shape)

# Low heterozygosity sites
strfile= "/storage/szfeupe/Runs/GTEx_estr/least_polymorphic_sites.tsv"
L_pol = pd.read_csv(strfile, sep='\t')
                #ID addition
L_pol['POS']= L_pol['POS'].astype('str')
L_pol['ID'] = L_pol['chrom']+'_'+L_pol['POS']
print('Regions of low heterozygosity: ',L_pol.shape)

# Linear regression output
linreg = pd.read_csv(Input+'/Lin_Reg_Out', sep='\t')
#linreg = pd.read_csv(Input+'/Lin_Reg_Out_perm', sep='\t')
                #ID addition
linreg['str.start']= linreg['str.start'].astype('str')
linreg['ID'] = linreg['chrom']+'_'+linreg['str.start']
print('All sites considered for LR: ',linreg.shape)

#-------
chrs = [i for i in range(1,23)]
chrs.append('X') ; chrs.append('Y')
#---------
index=['0']
Out = pd.DataFrame(index=index, columns=linreg.columns)
#------------


#LR Cleanup
for i in chrs:
#    print('..............chr',i)
    Reg= poly.loc[poly[0]=='chr'+str(i)]
    ID=list(Reg['ID']) ; IDs = []
 #Here we want to include any STR starting within 5bps of the start point
    for elt in ID:
        e = int(elt.split('_')[1])  ; x=elt.split('_')[0]
        f=[str(e), str(e+1), str(e+2),str(e+3),str(e+4),str(e+5), str(e-2),str(e-3),str(e-4),str(e-1)]   #+5bases
        f=[x+'_'+p for p in f]
        IDs += f  
 #List of IDs to consider completed    
    LIN = linreg.loc[linreg['chrom']=='chr'+str(i)]
    POL = L_pol.loc[L_pol['chrom']=='chr'+str(i)]
    
#Remove homopolymers regions from regression
    Filt1 = LIN.loc[LIN['ID'].isin(IDs)]     #>=Reg[1] and LIN['str.start']<=Reg[2]]

#Remove least polymorphic sites from regression
#least polymorphic sites (Het ziygosity <0.3)
    Filt2 = Filt1.loc[Filt1['ID'].isin(list(POL['ID']))==False]
#    print(LIN.shape, Filt1.shape,Filt2.shape[0])

#Output
    frames = [Out, Filt2]
    Out = pd.concat(frames)

Out1=Out.drop('0')
Out1=Out1.drop('Unnamed: 0', 1)
Out1.to_csv(Input+'/Lin_Reg_OutFin.txt', sep='\t', index=False)
Out1.shape

In [16]:
##Simpler version of FDR correction 
import pandas as pd
import numpy as np
import subprocess
import collections
def fdrcorrection(tissue):
    print(tissue, ' variants ...')
#Get most signif. variant by gene from linear reg STRs
    LR1=pd.read_csv("/storage/szfeupe/Runs/GTEx_estr/Analysis_by_Tissue/"+ tissue+ "/Lin_Reg_OutFin.txt", '\t')
    LR0 = LR1.sort("p.wald").groupby("gene", as_index=False).first()     
    print(LR1.shape, '  to  ', LR0.shape)

#Add counts by genes
    counts=pd.DataFrame({'cts' : LR1.groupby(["gene"]).size()})    ## This is the count by genes
    genes = list(LR0['gene'])
    LR0['NTEST']= list(counts.loc[genes]['cts'])
    print (len(counts.loc[genes]['cts']))
    
#Minor adjustment
    #(1) min_pval* #test
    LR0['AD.pval']=LR0['p.wald']*LR0['NTEST']
    #(2) if AD_pval>1 => AD_pval=1
    LR0['AD.pval'][LR0['AD.pval']>1] = 1
    
#Save pval in file and FDR correct
    LR0['AD.pval'].to_csv('pvalues.txt', sep='\n', index=False)
    Tell = subprocess.call("/home/szfeupe/projects/GTEX_eSTRs/gtex-estrs/Scripts/fdr-correct.r")
    
#FDR corrected... add to dataframe
    Qval=pd.read_csv('/home/szfeupe/projects/GTEX_eSTRs/gtex-estrs/Scripts/qvalues.txt', sep=' ')
    print (Qval.shape, '\t', LR0.shape)
    LR0['qvalue']=list(Qval['qvalue'])
    LR0['significant']=list(Qval['significant'])
    
#Header arrangement
    Head=['gene','chrom','str.id','str.start','NTEST','p.wald','AD.pval','qvalue','significant','beta','beta.se']
    Out=LR0[Head]
    Out.to_csv("/storage/szfeupe/Runs/GTEx_estr/Analysis_by_Tissue/"+tissue+'/PQValue.tsv', sep='\t', index=False)

    S=LR0['AD.pval']
    print(len(S),' total tests... ', len(S[S>=1]) , ' pvalues were reduced to 1\n')
    print(len(Qval[Qval['qvalue'] <=0.1]),'\t qval<=0.1')
    print(len(Qval[Qval['qvalue'] <=0.05]),'\t qval<=0.05')
    print(len(Qval[Qval['qvalue'] <=0.01]),'\t qval<0.01\n')
#   print('Original linear reg. tests counts\n',LR1.groupby('chrom').size())
#   print('After FDR correction test counts\n',LR0.groupby('chrom').size())    

    return()

fdrcorrection('Esophagus-Mucosa/SNP_Analysis')

Esophagus-Mucosa/SNP_Analysis  variants ...


/usr/lib/python3.4/site-packages/ipykernel/__main__.py:10: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


(10289721, 12)   to   (16211, 12)
16211


/usr/lib/python3.4/site-packages/ipykernel/__main__.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(16211, 6) 	 (16211, 14)
16211  total tests...  11413  pvalues were reduced to 1

691 	 qval<=0.1
556 	 qval<=0.05
379 	 qval<0.01



()

In [17]:
###This code prepares inputs for heritability test
import matplotlib.pyplot as plt
import pandas as pd
In = '/storage/szfeupe/Runs/GTEx_estr/Analysis_by_Tissue/Esophagus-Mucosa/'

#estrs
estr = pd.read_csv(In+'PQValue.tsv', sep="\t")
#esnps
esnp = pd.read_csv(In+'SNP_Analysis/PQValue.tsv', sep='\t')
#renaming columns
estr['qval.gene']=estr['qvalue']
esnp['qval.gene']=esnp['qvalue']
estr['pvalue']=estr['AD.pval']
esnp['pvalue']=esnp['AD.pval']
print(estr.shape, esnp.shape)

estr=estr.loc[estr['gene'].isin(list(esnp['gene']))]
print(estr.shape, esnp.shape, len(estr.loc[estr['qvalue']<=0.1]))

be=[1]*len(estr)
estr['best_str'] = be
be=[1]*len(esnp)
esnp['best_str'] = [1]*len(esnp)
Head=['gene', 'chrom', 'str.id', 'str.start', 'NTEST', 'beta','p.wald', 'best_str', 'qval.gene','significant','pvalue']
wbsnp=esnp.loc[:,Head]
wbstr=estr.loc[:,Head]
wbstr.to_csv(In+'HH/estr_results', sep='\t')
wbsnp.to_csv(In+'HH/esnp_results', sep='\t')
print(wbstr.shape)
wbsnp.shape

(15520, 13) (16211, 13)
(15520, 13) (16211, 13) 459
(15520, 11)


(16211, 11)

In [ ]:
#(182156, 14) out of 1346324 #--41333
import pandas as pd

Input='/storage/szfeupe/Runs/GTEx_estr/Analysis_by_Tissue/Artery-Tibial/' #CHANGE HERE!!!

# STR size unit file
Reg='/storage/szfeupe/data/Regions_HipSTR_reference.hg19.bed'
regions=pd.read_csv(Reg, sep='\t', header=None)
#ID addition
regions[1]= regions[1].astype('str')
regions['ID'] = regions[0]+'_'+ regions[1]
print(regions.shape)
check=pd.read_csv(Input+'/Lin_Reg_Out', sep='\t')
check['str.start']= check['str.start'].astype('int')
check['str.start']= check['str.start'].astype('str')
check['ID'] = check['chrom']+'_'+check['str.start']
print(check.shape)

Homop=regions.loc[regions[3]==1]
Homop[1]= Homop[1].astype('str')
Homop['ID'] = Homop[0]+'_'+ Homop[1]

print(Homop.shape)


Missed = check.loc[check['ID'].isin(list(regions['ID']))]
Missed.shape

In [1]:
##This code is specific for SNP data
##It will remove SNPs that fall in homopolymer and least polymorphic sites from linear regression analysis output
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import vcf
import os

Input='/storage/szfeupe/Runs/GTEx_estr/Analysis_by_Tissue/Cells-Transformedfibroblasts/SNP_Analysis/'    #Artery-Tibial/SNP_Analysis/'

linreg = pd.read_csv(Input+'/Lin_Reg_Out', sep='\t')
#linreg = pd.read_csv(Input+'/Lin_Reg_Out_perm', sep='\t')

print (Input, '\n',linreg.shape)

strfile= "/storage/szfeupe/Runs/GTEx_estr/least_polymorphic_sites.tsv"
L_pol = pd.read_csv(strfile, sep='\t')

chrs = [i for i in range(1,23)]
print(chrs)
chrs.append('X') ; chrs.append('Y')

index=['0']
Out = pd.DataFrame(index=index, columns=linreg.columns)

Reg='/storage/szfeupe/data/Regions_HipSTR_reference.hg19.bed'
regions=pd.read_csv(Reg, sep='\t', header=None)
poly=regions.loc[regions[3]==1]
#LR Cleanup
for i in chrs:
    
#   Remove Homoploymers
    Homopoly = poly.loc[poly[0]=='chr'+str(i)]
    LIN = linreg.loc[linreg['chrom']=='chr'+str(i)]
    FIL = LIN.loc[LIN['str.start'].isin(list(Homopoly[1]))==False]

#    Open least polymorphic sites (Het ziygosity <0.3)
    POL = L_pol.loc[L_pol['chrom']=='chr'+str(i)]
    Het = list(POL['POS'])
    Hets = [[e, e+1, e+2,e+3,e+4,e+5,e+6] for e in Het]
    Het = sum(Hets, [])
#    Remove those sites
    FIL_h = FIL.loc[FIL['str.start'].isin(Het)==False]

    
#   Sum it up and outout   
    frames = [Out, FIL_h]
    Out = pd.concat(frames)

    print(i,' Done ',LIN.shape,'\t', FIL.shape, '\t', FIL_h.shape,'\t',Out.shape)

Out1=Out.drop('0')
Out1=Out1.drop('Unnamed: 0', 1)
Out1.to_csv(Input+'/Lin_Reg_OutFin.txt', sep='\t', index=False)
#Out1.to_csv(Input+'/Lin_Reg_Out_permFinal.txt', sep='\t', index=False)  #869845---769487
print('End')

####After this Script, plot the qqplot and then FDR correction 

/storage/szfeupe/Runs/GTEx_estr/Analysis_by_Tissue/Cells-Transformedfibroblasts/SNP_Analysis/ 
 (9766340, 13)
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]
1  Done  (919571, 13) 	 (919543, 13) 	 (918379, 13) 	 (918380, 13)
2  Done  (668690, 13) 	 (668670, 13) 	 (667918, 13) 	 (1586298, 13)
3  Done  (595685, 13) 	 (595666, 13) 	 (595070, 13) 	 (2181368, 13)
4  Done  (456948, 13) 	 (456939, 13) 	 (456507, 13) 	 (2637875, 13)
5  Done  (475851, 13) 	 (475846, 13) 	 (475453, 13) 	 (3113328, 13)
6  Done  (652426, 13) 	 (652419, 13) 	 (651720, 13) 	 (3765048, 13)
7  Done  (511925, 13) 	 (511912, 13) 	 (511334, 13) 	 (4276382, 13)
8  Done  (395173, 13) 	 (395158, 13) 	 (394721, 13) 	 (4671103, 13)
9  Done  (407115, 13) 	 (407108, 13) 	 (406739, 13) 	 (5077842, 13)
10  Done  (417027, 13) 	 (417017, 13) 	 (416491, 13) 	 (5494333, 13)
11  Done  (531433, 13) 	 (531414, 13) 	 (530812, 13) 	 (6025145, 13)
12  Done  (523343, 13) 	 (523320, 13) 	 (522679, 13) 	 (65478